### 停用词

In [5]:
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords
# segs = filter(lambda x: x not in stopwords, segs)  # 去掉停用词

In [6]:
stopwords = stopwordslist("/opt/project/Jupyter/StaticFile/stopwords.txt")

### 分词

In [ ]:
# https://github.com/fxsjy/jieba
# jieba.add_word("不可抗力",99,"nz")
# list(jieba.cut("南凌科技股份有限公司创业板首次公开发行股票招股说明书（申报稿2019年10月23日报送）"))

# jieba.add_word("股份有限公司",1,"nz")
# list(jieba.cut("南凌科技股份有限公司创业板首次公开发行股票招股说明书（申报稿2019年10月23日报送）",HMM=True))
import jieba

In [41]:
def segment(content):
    """
    单line分词
    """
    if "：" in content:
        result = (str(content.split("：")[1])).replace("\r\n", "").strip()  # 删除多余空行与空格
    else:
        result = content
    cutResult = jieba.lcut(result)  # 默认方式分词，分词结果用空格隔开
    return " ".join(cutResult)


def batch_segment(lines):
    """
    批量lines分词
    """
    seg_lines = []
    for line in lines:
        seg_lines.append(segment(line))
    return seg_lines

### tfidf

In [7]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [34]:
def create_tfidfvector(vocab=None,max_feature=500):
    if vocab:
        return TfidfVectorizer(stopwords,
                                           smooth_idf=True,
                                           token_pattern=r"(?u)\b\w+\b",
                                           max_features=max_feature,
                                           vocabulary=vocab 
                                          )
    else:
        return TfidfVectorizer(stopwords,
#                                            smooth_idf=True,
                                           token_pattern=r"(?u)\b\w+\b",
                                           max_features=max_feature,
                                          )
tfidf_vectorizer = create_tfidfvector()
texts = ["我今 北京 下雨了", "我今 加班"]
tfidf_vectorizer.fit_transform(texts)                             

<2x4 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [24]:
tfidf_vectorizer.vocabulary_  # 词表：词对应的index

{'我今': 3, '北京': 2, '下雨了': 0, '加班': 1}

In [31]:
tfidf_vectorizer.get_feature_names()

['下雨了', '加班', '北京', '我今']

In [29]:
# 
tfidf_vectorizer.transform([texts[0]]).toarray()  # 

array([[0.6316672 , 0.        , 0.6316672 , 0.44943642]])

In [30]:
tfidf_vectorizer.transform([texts[1]]).toarray()

array([[0.        , 0.81480247, 0.        , 0.57973867]])

In [38]:
# CountVectorizer?
def create_countvector(vocab,max_feature=500):
        return CountVectorizer(stopwords,
                                vocabulary=vocab,
                               token_pattern=r"(?u)\b\w+\b",
                               max_features=max_feature,
                                      )

### 字/词向量

In [6]:
import codecs
import numpy as np

In [7]:
def load_pretrained_vec(pretrained_path,vec_len):
    pre_trained = {}
    for i, line in enumerate(codecs.open(pretrained_path, 'r', 'utf-8')):
        line = line.rstrip().split()
        # print(line)
        if len(line) == vec_len + 1:
            pre_trained[line[0]] = np.array(
                [float(x) for x in line[1:]]
            ).astype(np.float32)
    return pre_trained

In [8]:
char_pre_trained = load_pretrained_vec("/opt/project/Jupyter/StaticFile/char_wiki_100.utf8",vec_len=100)

In [23]:
# word_pre_trained = load_pretrained_vec("/opt/project/Jupyter/StaticFile/sgns.financial.word",vec_len=300)

In [26]:
# char_pre_trained['一']

### 关键词提取

In [13]:
import jieba
import jieba.analyse
import jieba.posseg as pseg

In [14]:
allow_poses = ('ns', 'n', 'nz','nrt','vn', 'v','t','tg','tg','qt','mq','m','nr','nt',"vn",'q','l')
# 关键词提取：基于TF-IDF:
def get_keywords(content):
    """
    获取关键词
    去停用词
    """
    keywords = jieba.analyse.extract_tags(
                                    sentence=content,
                                    topK=20,   
                                    withWeight=True,
                                    allowPOS=allow_poses
                                    )
    return keywords
keywords = get_keywords("第9条 合同生效、变更和解除")
keywords

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.019 seconds.
Prefix dict has been built succesfully.


[('生效', 2.552170250436667),
 ('解除', 2.4304428029366667),
 ('变更', 2.2520901572166667)]

### 短语/句子/段落的向量化

In [16]:
def resize_weight(keywords):
    """
    归一化权重
    """
#     print(type(keywords))
    a = 0.0
    new_ks = []
    for k in keywords:
#         print(k)
        a +=k[1]
    for k in keywords:
        new_ks.append((k[0],k[1]/a))
    return new_ks

def weight_char_embed(chars, top_k=5,vec_len=100):
    """
    短语向量表示：加权词向量
    """
    vec = np.zeros((vec_len))
    chars = chars.replace(" ", "")
    keywords = get_keywords(chars)
    keywords = keywords[0:top_k]
#     print(keywords)
    keywords = resize_weight(keywords)
#     print(keywords)
    for keyword in keywords:
        for char in keyword[0]:
            if char in char_pre_trained.keys():
                vec +=char_pre_trained[char]*keyword[1]
    return vec

def batch_wce(lines):
    return list(map(weight_char_embed,lines))

In [18]:
# chars = "信息检索的PR曲线（Precision-Recall Curve）作图"
# weight_char_embed(chars)

In [ ]:
def get_inter_vocabs(pretrained_vocabs,text_vocab):
    """
    词向量字典和文本字典的交集
    """
    vocab = pretrained_vocabs & text_vocab
    return vocab

In [3]:
8*1000*0.006

48.0

In [1]:
from keras.preprocessing.text import Tokenizer

def corpus2vocab_keras(lines):
    """
    lines: list,sentences
    """
    seg_lines = batch_segment(lines)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(seg_lines)
    # `0` is a reserved index that won't be assigned to any word.
    return tokenizer
# tokenizer.word_index.keys() # text_vocab
# len(tokenizer.word_counts)

Using TensorFlow backend.


In [2]:
# keras 词频统计
text = ["今天北京下雨了", "我今 加班"]
corpus2vocab_keras(text).word_counts

NameError: name 'batch_segment' is not defined

In [3]:
def sent2seq(sent,tokenizer):
    """
    func:句子生成word_index序列
    sent: sentence,str
    """
    texts = [segment(sent)]
    return keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(texts), 
                                           maxlen=100,
                                           padding="post",
                                           truncating="post")[0]
# sent2seq("我们乙方根据甲方的委托承包甲方指定的生产线项目，并与甲方共同选派作业人员，在遵守甲方的有关")

In [46]:
# -*- cobert_encg:utf-8 -*-
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity


class Encoding(object):
    def __init__(self):
        self.server_ip = "127.0.0.1"
        self.bert_client = BertClient(ip=self.server_ip,check_length=False)

    def encode(self, query):
        tensor = self.bert_client.encode([query])
        return tensor
    
    def encodes(self, queries):
        tensors = self.bert_client.encode(queries)
        return tensors

    def query_similarity(self, query_list):
        tensors = self.bert_client.encode(query_list)
        return cosine_similarity(tensors)[0][1]
    
ec = Encoding()
def bert_enc(sents):
    def isempty(sent):
        if not sent or sent.strip() == "":
            return "其它"
        else:
            return sent
    sents = list(map(isempty,sents))
    return ec.encodes(sents)


In [11]:
round(0.7)

1

In [1]:
"" == None

False